## Generate feature extracted data based on window size, overlap and class numbers.
1. Extract features with three class labels

In [37]:
import sys
sys.path.append('/home/edumaba/Public/MPhil_Thesis/Code/wear_uropatch')
import os
import pickle
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from essentials import three_class_labels, normalize
from essentials_improved import complete_preprocessing_pipeline
from feature_set_2_minus_time_energy import GenerateFeatures 
import copy

In [38]:
with open('/home/edumaba/Public/MPhil_Thesis/Code/wear_uropatch/data/df_dict_imu.pkl', 'rb') as f:
    imu_dict = pickle.load(f)
with open('/home/edumaba/Public/MPhil_Thesis/Code/wear_uropatch/data/df_minze_dict.pkl', 'rb') as f:
    ground_truth_dict = pickle.load(f)
with open('/home/edumaba/Public/MPhil_Thesis/Code/wear_uropatch/data/df_dict_urineestimate_method1.pkl', 'rb') as f:
    urine_estimate_dict = pickle.load(f)

Remove the data with the almost the entire data being void.
1. subj_9_void4
2. subj_11_void2

In [39]:
del imu_dict['subj_9_void4']
del imu_dict['subj_11_void2']

Extract windowed features from the 3 axis of the accelerometer and the acceleration magnitude.
1. mean
2. rms
3. std
4. range
5. variance
6. min
7. max
8. time energy
9. spectral energy
10. permutation entropy
11. spectral entropy
12. spectral_centroid
13. spectral_spread
14. spectral_flatness
15. peak_frequency_ratio
16. hjorth_frequency
17. spectral_skewness
18. spectral_kurtosis
19. snr


## Three class data

## Data 6
classes: pre-void, void, non-void

fs: original

overlap: 0.0

sliding window: 1s

filename: three_class_raw_1s_no.csv

In [40]:
# labelled_imu_dict = {}
# dict = copy.deepcopy(imu_dict)
# for i_void_instance, void_instance in tqdm(enumerate(dict.keys()), desc="Adding labels to IMU data"):
#     acc = dict[void_instance]
#     gt = ground_truth_dict[void_instance]
    
#     # normalize data
#     df = normalize(acc)
    
#     # add the labels
#     labelled_df = three_class_labels(df, gt)
    
#     labelled_imu_dict[void_instance] = labelled_df

In [41]:
data_dict = copy.deepcopy(imu_dict)
labelled_imu_dict = complete_preprocessing_pipeline(data_dict, ground_truth_dict, 
                                target_fs=50, remove_gravity=True, 
                                normalize_data=True, use_three_classes=True)

Step 1: Resampling data to 50 Hz...


  0%|          | 0/41 [00:00<?, ?it/s]

Step 2: Processing each instance: 100%|██████████| 41/41 [00:00<00:00, 136.01it/s]


In [42]:
all_features = []
for exp_id, imu_data in enumerate(labelled_imu_dict.keys()):    
    # Extract features
    
    df = labelled_imu_dict[imu_data]    
    analyzer = GenerateFeatures(50, window_duration=1.0, overlap=0.0)
    features, _ = analyzer.analyze_multi_axis_imu(labelled_imu_dict[imu_data])

    table = analyzer.create_summary_table()
    table['experiment_id'] = exp_id + 1  # Track source
    all_features.append(table)
    

final_features = pd.concat(all_features, ignore_index=True)

Analyzing : 100%|██████████| 8/8 [00:00<00:00, 14.59it/s]


In [43]:
# with pd.option_context('display.max_rows', None,
#                        'display.max_columns', None,
#                        'display.precision', 3,
#                        ):
#     print(final_features)
# # final_features

In [44]:
final_features.to_csv('/home/edumaba/Public/MPhil_Thesis/Code/wear_uropatch/feature_datasets/subject_based_norm/preprocessed/feature_set_2/without_gravity/three_class_pp_1s_no.csv', index=False)
final_features

,center_time,start_time,end_time,acc_x_permutation_entropy,acc_x_spectral_entropy,acc_x_mean,acc_x_std,acc_x_range,acc_x_rms,acc_x_var,...,gyr_mag_spectral_centroid,gyr_mag_spectral_spread,gyr_mag_spectral_flatness,gyr_mag_peak_frequency_ratio,gyr_mag_hjorth_frequency,gyr_mag_spectral_skewness,gyr_mag_spectral_kurtosis,gyr_mag_snr,label,experiment_id
0,0.5,0.0,1.0,0.688102,0.514527,-0.404491,2.084222,9.328774,2.123110,4.343982,...,5.057028,2.514687,0.042563,0.240618,6.898871,0.360728,5.079209,-4.991317,pre-void,1
1,1.5,1.0,2.0,0.820452,0.565028,0.371080,1.523115,6.167925,1.567667,2.319880,...,2.928394,1.832371,0.013403,0.352450,6.210352,1.492127,9.667187,-2.641750,pre-void,1
2,2.5,2.0,3.0,0.856360,0.662879,-0.048401,0.410007,1.652187,0.412854,0.168106,...,5.452486,4.044970,0.241426,0.199986,10.803375,1.104519,4.661007,-6.020985,pre-void,1
3,3.5,3.0,4.0,0.838706,0.723939,0.052971,0.489006,2.367645,0.491867,0.239127,...,2.738796,2.885323,0.080532,0.515280,8.665922,2.561354,11.288697,0.265528,pre-void,1
4,4.5,4.0,5.0,0.892830,0.730502,0.196923,0.353868,1.979306,0.404970,0.125222,...,3.109948,2.876128,0.082769,0.312278,9.778855,2.611918,11.027998,-3.428717,pre-void,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2393,36.5,36.0,37.0,0.776239,0.768316,0.255913,0.290046,1.479244,0.386805,0.084127,...,2.890789,3.933639,0.103298,0.683033,10.337058,2.254719,7.628977,3.334271,post-void,41
2394,37.5,37.0,38.0,0.881010,0.726027,-0.224245,0.409821,1.553701,0.467161,0.167954,...,2.503290,2.414863,0.061524,0.436803,8.546195,3.362574,18.114688,-1.103740,post-void,41
2395,38.5,38.0,39.0,0.873283,0.775359,0.203643,0.371366,1.468320,0.423536,0.137913,...,7.435864,4.806895,0.206111,0.158455,11.196329,0.921133,2.529281,-7.251709,post-void,41
2396,39.5,39.0,40.0,0.848737,0.650432,0.076882,0.410546,1.623017,0.417683,0.168548,...,5.190489,3.676363,0.153966,0.240182,8.936147,1.636480,5.739390,-5.001687,post-void,41


## Data 6b
classes: pre-void, void, non-void

fs: original

overlap: 0.5

sliding window: 1s

filename: three_class_raw_1s_yo_0.5.csv

In [45]:
all_features = []
for exp_id, imu_data in enumerate(labelled_imu_dict.keys()):    
    # Extract features
    
    df = labelled_imu_dict[imu_data]
    analyzer = GenerateFeatures(50, window_duration=1.0, overlap=0.5)
    features, _ = analyzer.analyze_multi_axis_imu(labelled_imu_dict[imu_data])

    table = analyzer.create_summary_table()
    table['experiment_id'] = exp_id + 1  # Track source
    all_features.append(table)
    

final_features = pd.concat(all_features, ignore_index=True)

Analyzing : 100%|██████████| 8/8 [00:01<00:00,  6.83it/s]


In [46]:
final_features.to_csv('/home/edumaba/Public/MPhil_Thesis/Code/wear_uropatch/feature_datasets/subject_based_norm/preprocessed/feature_set_2/without_gravity/three_class_pp_1s_0.5.csv', index=False)
final_features

,center_time,start_time,end_time,acc_x_permutation_entropy,acc_x_spectral_entropy,acc_x_mean,acc_x_std,acc_x_range,acc_x_rms,acc_x_var,...,gyr_mag_spectral_centroid,gyr_mag_spectral_spread,gyr_mag_spectral_flatness,gyr_mag_peak_frequency_ratio,gyr_mag_hjorth_frequency,gyr_mag_spectral_skewness,gyr_mag_spectral_kurtosis,gyr_mag_snr,label,experiment_id
0,0.5,0.0,1.0,0.688102,0.514527,-0.404491,2.084222,9.328774,2.123110,4.343982,...,5.057028,2.514687,0.042563,0.240618,6.898871,0.360728,5.079209,-4.991317,pre-void,1
1,1.0,0.5,1.5,0.738226,0.575262,0.435971,2.263858,9.328774,2.305455,5.125053,...,4.516458,2.304227,0.043312,0.212938,6.849813,1.460814,7.738428,-5.677551,pre-void,1
2,1.5,1.0,2.0,0.820452,0.565028,0.371080,1.523115,6.167925,1.567667,2.319880,...,2.928394,1.832371,0.013403,0.352450,6.210352,1.492127,9.667187,-2.641750,pre-void,1
3,2.0,1.5,2.5,0.845799,0.697468,-0.265863,0.540882,2.643078,0.602692,0.292554,...,3.256222,4.227513,0.211445,0.620694,7.719289,2.300055,7.981280,2.138874,pre-void,1
4,2.5,2.0,3.0,0.856360,0.662879,-0.048401,0.410007,1.652187,0.412854,0.168106,...,5.452486,4.044970,0.241426,0.199986,10.803375,1.104519,4.661007,-6.020985,pre-void,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4777,38.5,38.0,39.0,0.873283,0.775359,0.203643,0.371366,1.468320,0.423536,0.137913,...,7.435864,4.806895,0.206111,0.158455,11.196329,0.921133,2.529281,-7.251709,post-void,41
4778,39.0,38.5,39.5,0.849364,0.561674,0.007866,0.298129,1.237896,0.298232,0.088881,...,4.125308,3.276939,0.128552,0.364623,10.035662,2.157558,8.026538,-2.411874,post-void,41
4779,39.5,39.0,40.0,0.848737,0.650432,0.076882,0.410546,1.623017,0.417683,0.168548,...,5.190489,3.676363,0.153966,0.240182,8.936147,1.636480,5.739390,-5.001687,post-void,41
4780,40.0,39.5,40.5,0.872713,0.638198,-0.064653,0.538685,1.972815,0.542551,0.290181,...,5.762892,3.354067,0.186210,0.141004,9.045702,0.771545,4.898694,-7.847612,post-void,41


## Data 6c
classes: pre-void, void, non-void

fs: original

overlap: 0.8

sliding window: 1s

filename: three_class_raw_1s_yo_0.8.csv

In [47]:
all_features = []
for exp_id, imu_data in enumerate(labelled_imu_dict.keys()):    
    # Extract features
    
    df = labelled_imu_dict[imu_data]
    analyzer = GenerateFeatures(50, window_duration=1.0, overlap=0.8)
    features, _ = analyzer.analyze_multi_axis_imu(labelled_imu_dict[imu_data])

    table = analyzer.create_summary_table()
    table['experiment_id'] = exp_id + 1  # Track source
    all_features.append(table)
    

final_features = pd.concat(all_features, ignore_index=True)

Analyzing : 100%|██████████| 8/8 [00:02<00:00,  2.70it/s]


In [48]:
final_features.to_csv('/home/edumaba/Public/MPhil_Thesis/Code/wear_uropatch/feature_datasets/subject_based_norm/preprocessed/feature_set_2/without_gravity/three_class_pp_1s_0.8.csv', index=False)
final_features

,center_time,start_time,end_time,acc_x_permutation_entropy,acc_x_spectral_entropy,acc_x_mean,acc_x_std,acc_x_range,acc_x_rms,acc_x_var,...,gyr_mag_spectral_centroid,gyr_mag_spectral_spread,gyr_mag_spectral_flatness,gyr_mag_peak_frequency_ratio,gyr_mag_hjorth_frequency,gyr_mag_spectral_skewness,gyr_mag_spectral_kurtosis,gyr_mag_snr,label,experiment_id
0,0.5,0.0,1.0,0.688102,0.514527,-0.404491,2.084222,9.328774,2.123110,4.343982,...,5.057028,2.514687,0.042563,0.240618,6.898871,0.360728,5.079209,-4.991317,pre-void,1
1,0.7,0.2,1.2,0.723164,0.520272,-0.192136,2.119607,9.328774,2.128298,4.492735,...,3.759596,2.914030,0.058487,0.293138,6.866987,1.554107,6.970434,-3.822629,pre-void,1
2,0.9,0.4,1.4,0.738226,0.514491,0.380313,2.225862,9.328774,2.258119,4.954462,...,4.446676,2.599471,0.060663,0.213847,7.169435,1.150370,6.368941,-5.654047,pre-void,1
3,1.1,0.6,1.6,0.749550,0.558666,0.560449,2.176849,9.328774,2.247838,4.738672,...,4.350862,2.055888,0.040136,0.341086,6.691256,2.164400,10.672378,-2.859645,pre-void,1
4,1.3,0.8,1.8,0.753356,0.602276,0.413008,2.220996,9.328774,2.259070,4.932824,...,3.927820,1.885111,0.035920,0.440220,6.466994,2.245437,12.751827,-1.043474,pre-void,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11912,39.7,39.2,40.2,0.850044,0.686834,-0.087761,0.553451,1.972815,0.560366,0.306308,...,5.022787,3.364935,0.170049,0.222364,9.072008,1.835288,7.221196,-5.437129,post-void,41
11913,39.9,39.4,40.4,0.851570,0.626797,-0.096529,0.534306,1.972815,0.542956,0.285483,...,5.379003,3.173508,0.173299,0.199150,9.179031,1.207460,6.275119,-6.043715,post-void,41
11914,40.1,39.6,40.6,0.873432,0.686613,-0.068044,0.542963,1.972815,0.547210,0.294809,...,5.986334,3.633234,0.166671,0.166066,8.655420,0.709520,4.107200,-7.008515,post-void,41
11915,40.3,39.8,40.8,0.871095,0.652800,-0.349453,0.566234,2.073913,0.665387,0.320621,...,5.321478,3.471453,0.124318,0.325412,8.542409,1.393359,5.157608,-3.166052,post-void,41


## Data 7
classes: pre-void, void, non-void

fs: original

overlap: 0.0

sliding window: 2s

filename: three_class_raw_2s_no.csv

In [49]:
all_features = []
for exp_id, imu_data in enumerate(labelled_imu_dict.keys()):    
    # Extract features
    
    df = labelled_imu_dict[imu_data]
    analyzer = GenerateFeatures(50, window_duration=2.0, overlap=0.0)
    features, _ = analyzer.analyze_multi_axis_imu(labelled_imu_dict[imu_data])

    table = analyzer.create_summary_table()
    table['experiment_id'] = exp_id + 1  # Track source
    all_features.append(table)
    

final_features = pd.concat(all_features, ignore_index=True)

Analyzing : 100%|██████████| 8/8 [00:00<00:00, 28.92it/s]


In [50]:
final_features.to_csv('/home/edumaba/Public/MPhil_Thesis/Code/wear_uropatch/feature_datasets/subject_based_norm/preprocessed/feature_set_2/without_gravity/three_class_pp_2s_no.csv', index=False)
final_features

,center_time,start_time,end_time,acc_x_permutation_entropy,acc_x_spectral_entropy,acc_x_mean,acc_x_std,acc_x_range,acc_x_rms,acc_x_var,...,gyr_mag_spectral_centroid,gyr_mag_spectral_spread,gyr_mag_spectral_flatness,gyr_mag_peak_frequency_ratio,gyr_mag_hjorth_frequency,gyr_mag_spectral_skewness,gyr_mag_spectral_kurtosis,gyr_mag_snr,label,experiment_id
0,1.0,0.0,2.0,0.773704,0.674986,-0.016706,1.866095,9.328774,1.866169,3.482309,...,3.892970,2.404412,0.044606,0.136661,6.720479,1.381339,7.587279,-8.005353,pre-void,1
1,3.0,2.0,4.0,0.840974,0.775521,0.002285,0.454076,2.367645,0.454082,0.206185,...,3.898506,4.393158,0.188524,0.341835,9.276065,1.564536,5.239449,-2.845178,pre-void,1
2,5.0,4.0,6.0,0.864127,0.752471,-0.137509,0.885444,6.274071,0.896058,0.784011,...,1.903628,2.569054,0.055822,0.489318,7.346493,3.378154,17.205281,-0.185599,pre-void,1
3,7.0,6.0,8.0,0.876542,0.712354,0.285113,1.525026,9.255942,1.551449,2.325703,...,4.551480,3.570949,0.101051,0.127263,9.024110,1.010299,4.465060,-8.361804,pre-void,1
4,9.0,8.0,10.0,0.860081,0.749784,-0.107636,0.764003,4.947701,0.771548,0.583701,...,2.148733,2.993990,0.068257,0.408667,8.683691,2.927533,12.273310,-1.604630,pre-void,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1183,31.0,30.0,32.0,0.899131,0.725184,-0.511437,1.571397,6.215894,1.652531,2.469289,...,1.949664,2.499379,0.024572,0.503360,7.871503,2.543878,10.636591,0.058369,post-void,41
1184,33.0,32.0,34.0,0.838170,0.803764,0.385103,0.909225,3.989503,0.987418,0.826690,...,2.144783,3.116440,0.057387,0.506940,8.848575,2.285773,8.274812,0.120572,post-void,41
1185,35.0,34.0,36.0,0.869228,0.648455,-0.083578,0.820064,3.582379,0.824312,0.672506,...,1.014721,1.632677,0.018112,0.594529,8.857330,6.621829,54.192137,1.662140,post-void,41
1186,37.0,36.0,38.0,0.844912,0.829470,0.015834,0.428577,1.881228,0.428869,0.183678,...,2.155708,2.572583,0.055189,0.277348,8.712406,3.167330,16.048487,-4.159047,post-void,41


## Data 7b
classes: pre-void, void, non-void

fs: original

overlap: 0.5

sliding window: 2s

filename: three_class_raw_2s_yo_0.5.csv

In [51]:
all_features = []
for exp_id, imu_data in enumerate(labelled_imu_dict.keys()):    
    # Extract features
    
    df = labelled_imu_dict[imu_data]
    analyzer = GenerateFeatures(50, window_duration=2.0, overlap=0.5)
    features, _ = analyzer.analyze_multi_axis_imu(labelled_imu_dict[imu_data])

    table = analyzer.create_summary_table()
    table['experiment_id'] = exp_id + 1  # Track source
    all_features.append(table)
    

final_features = pd.concat(all_features, ignore_index=True)

Analyzing : 100%|██████████| 8/8 [00:00<00:00, 14.85it/s]


In [52]:
final_features.to_csv('/home/edumaba/Public/MPhil_Thesis/Code/wear_uropatch/feature_datasets/subject_based_norm/preprocessed/feature_set_2/without_gravity/three_class_pp_2s_0.5.csv', index=False)
final_features

,center_time,start_time,end_time,acc_x_permutation_entropy,acc_x_spectral_entropy,acc_x_mean,acc_x_std,acc_x_range,acc_x_rms,acc_x_var,...,gyr_mag_spectral_centroid,gyr_mag_spectral_spread,gyr_mag_spectral_flatness,gyr_mag_peak_frequency_ratio,gyr_mag_hjorth_frequency,gyr_mag_spectral_skewness,gyr_mag_spectral_kurtosis,gyr_mag_snr,label,experiment_id
0,1.0,0.0,2.0,0.773704,0.674986,-0.016706,1.866095,9.328774,1.866169,3.482309,...,3.892970,2.404412,0.044606,0.136661,6.720479,1.381339,7.587279,-8.005353,pre-void,1
1,2.0,1.0,3.0,0.844456,0.625148,0.161340,1.134894,6.167925,1.146305,1.287984,...,1.392502,1.825577,0.029816,0.574225,6.532965,4.382557,32.983024,1.299024,pre-void,1
2,3.0,2.0,4.0,0.840974,0.775521,0.002285,0.454076,2.367645,0.454082,0.206185,...,3.898506,4.393158,0.188524,0.341835,9.276065,1.564536,5.239449,-2.845178,pre-void,1
3,4.0,3.0,5.0,0.868150,0.746353,0.124947,0.432845,2.367645,0.450518,0.187355,...,3.059343,2.822088,0.075571,0.173558,9.110175,2.228843,9.153422,-6.777675,pre-void,1
4,5.0,4.0,6.0,0.864127,0.752471,-0.137509,0.885444,6.274071,0.896058,0.784011,...,1.903628,2.569054,0.055822,0.489318,7.346493,3.378154,17.205281,-0.185599,pre-void,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2352,36.0,35.0,37.0,0.886129,0.806328,0.060338,0.581728,2.576756,0.584848,0.338407,...,3.246206,3.997926,0.123924,0.376860,10.512525,2.131183,6.931733,-2.184054,post-void,41
2353,37.0,36.0,38.0,0.844912,0.829470,0.015834,0.428577,1.881228,0.428869,0.183678,...,2.155708,2.572583,0.055189,0.277348,8.712406,3.167330,16.048487,-4.159047,post-void,41
2354,38.0,37.0,39.0,0.888111,0.732971,-0.010301,0.445764,1.866043,0.445883,0.198705,...,1.950713,2.529798,0.040635,0.325026,9.416002,3.318885,16.130658,-3.173691,post-void,41
2355,39.0,38.0,40.0,0.868384,0.744292,0.140262,0.396545,1.623017,0.420620,0.157248,...,4.758725,4.083044,0.133004,0.261959,10.057943,1.480881,4.748336,-4.498467,post-void,41


## Data 7c
classes: pre-void, void, non-void

fs: original

overlap: 0.8

sliding window: 2s

filename: three_class_raw_2s_yo_0.8.csv

In [53]:
all_features = []
for exp_id, imu_data in enumerate(labelled_imu_dict.keys()):    
    # Extract features
    
    df = labelled_imu_dict[imu_data]
    analyzer = GenerateFeatures(50, window_duration=2.0, overlap=0.8)
    features, _ = analyzer.analyze_multi_axis_imu(labelled_imu_dict[imu_data])

    table = analyzer.create_summary_table()
    table['experiment_id'] = exp_id + 1  # Track source
    all_features.append(table)
    

final_features = pd.concat(all_features, ignore_index=True)

Analyzing : 100%|██████████| 8/8 [00:01<00:00,  5.52it/s]


In [54]:
final_features.to_csv('/home/edumaba/Public/MPhil_Thesis/Code/wear_uropatch/feature_datasets/subject_based_norm/preprocessed/feature_set_2/without_gravity/three_class_pp_2s_0.8.csv', index=False)
final_features

,center_time,start_time,end_time,acc_x_permutation_entropy,acc_x_spectral_entropy,acc_x_mean,acc_x_std,acc_x_range,acc_x_rms,acc_x_var,...,gyr_mag_spectral_centroid,gyr_mag_spectral_spread,gyr_mag_spectral_flatness,gyr_mag_peak_frequency_ratio,gyr_mag_hjorth_frequency,gyr_mag_spectral_skewness,gyr_mag_spectral_kurtosis,gyr_mag_snr,label,experiment_id
0,1.0,0.0,2.0,0.773704,0.674986,-0.016706,1.866095,9.328774,1.866169,3.482309,...,3.892970,2.404412,0.044606,0.136661,6.720479,1.381339,7.587279,-8.005353,pre-void,1
1,1.4,0.4,2.4,0.789930,0.664100,0.163226,1.694064,9.328774,1.701909,2.869852,...,2.675904,2.185320,0.021427,0.283998,7.089577,1.474120,7.785894,-4.015988,pre-void,1
2,1.8,0.8,2.8,0.839647,0.674704,0.170606,1.614078,9.328774,1.623070,2.605248,...,1.640867,1.766996,0.019691,0.478500,6.659953,2.714439,17.072805,-0.373728,pre-void,1
3,2.2,1.2,3.2,0.845633,0.638755,0.112239,1.034637,6.167925,1.040707,1.070473,...,1.548048,2.616764,0.074072,0.678450,6.558597,4.014341,22.472755,3.242692,pre-void,1
4,2.6,1.6,3.6,0.850864,0.766906,-0.080774,0.463914,2.643078,0.470894,0.215216,...,4.662815,4.509131,0.240550,0.174967,9.875033,1.378437,4.846663,-6.735149,pre-void,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5862,38.2,37.2,39.2,0.892014,0.694349,0.004375,0.439292,1.866043,0.439314,0.192978,...,2.146112,3.142142,0.054827,0.430868,9.362995,2.894136,11.814651,-1.208680,post-void,41
5863,38.6,37.6,39.6,0.879039,0.736495,0.064944,0.406719,1.866043,0.411871,0.165420,...,3.928902,4.623132,0.122136,0.438571,9.423344,1.496074,4.350653,-1.072550,post-void,41
5864,39.0,38.0,40.0,0.868384,0.744292,0.140262,0.396545,1.623017,0.420620,0.157248,...,4.758725,4.083044,0.133004,0.261959,10.057943,1.480881,4.748336,-4.498467,post-void,41
5865,39.4,38.4,40.4,0.870491,0.673541,-0.017617,0.454914,1.972815,0.455255,0.206947,...,4.340463,3.541303,0.130643,0.181037,9.836597,1.699885,6.595340,-6.554960,post-void,41


## Data 8
classes: pre-void, void, non-void

fs: original

overlap: 0.0

sliding window: 3s

filename: three_class_raw_3s_no.csv

In [55]:
all_features = []
for exp_id, imu_data in enumerate(labelled_imu_dict.keys()):    
    # Extract features
    
    df = labelled_imu_dict[imu_data] 
    analyzer = GenerateFeatures(50, window_duration=3.0, overlap=0.0)
    features, _ = analyzer.analyze_multi_axis_imu(labelled_imu_dict[imu_data])

    table = analyzer.create_summary_table()
    table['experiment_id'] = exp_id + 1  # Track source
    all_features.append(table)
    

final_features = pd.concat(all_features, ignore_index=True)

Analyzing : 100%|██████████| 8/8 [00:00<00:00, 40.86it/s]


In [56]:
final_features.to_csv('/home/edumaba/Public/MPhil_Thesis/Code/wear_uropatch/feature_datasets/subject_based_norm/preprocessed/feature_set_2/without_gravity/three_class_pp_3s_no.csv', index=False)
final_features

,center_time,start_time,end_time,acc_x_permutation_entropy,acc_x_spectral_entropy,acc_x_mean,acc_x_std,acc_x_range,acc_x_rms,acc_x_var,...,gyr_mag_spectral_centroid,gyr_mag_spectral_spread,gyr_mag_spectral_flatness,gyr_mag_peak_frequency_ratio,gyr_mag_hjorth_frequency,gyr_mag_spectral_skewness,gyr_mag_spectral_kurtosis,gyr_mag_snr,label,experiment_id
0,1.5,0.0,3.0,0.810113,0.693263,-0.027271,1.542011,9.328774,1.542252,2.377798,...,2.217571,2.239953,0.023149,0.288816,6.824659,1.703263,8.248314,-3.913610,pre-void,1
1,4.5,3.0,6.0,0.860940,0.765878,-0.074015,0.781310,6.274071,0.784808,0.610446,...,2.473462,2.888966,0.062104,0.219981,7.693017,2.606673,11.146248,-5.497193,pre-void,1
2,7.5,6.0,9.0,0.864884,0.716930,0.159788,1.267820,9.255942,1.277850,1.607368,...,4.576898,3.476976,0.104631,0.098653,8.897261,1.152538,4.852549,-9.607808,pre-void,1
3,10.5,9.0,12.0,0.853932,0.737436,0.062877,0.859815,4.947701,0.862111,0.739282,...,1.819047,2.522506,0.044793,0.434533,8.412370,2.797448,13.341614,-1.143843,pre-void,1
4,13.5,12.0,15.0,0.882423,0.747484,0.042560,0.803986,5.867749,0.805112,0.646393,...,3.317767,4.134981,0.135966,0.136892,10.529129,2.415791,9.412629,-7.996888,pre-void,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
781,25.5,24.0,27.0,0.854082,0.768725,0.058473,0.558995,3.155046,0.562045,0.312475,...,2.542707,3.066651,0.082837,0.229679,8.813088,2.230269,9.711518,-5.255495,void,41
782,28.5,27.0,30.0,0.871580,0.804707,0.097366,0.929788,4.441176,0.934872,0.864506,...,3.737058,3.202922,0.082731,0.096852,8.859343,1.435611,5.529354,-9.696526,post-void,41
783,31.5,30.0,33.0,0.884866,0.679333,-0.114512,1.544676,7.404909,1.548915,2.386025,...,1.435746,2.088406,0.015275,0.445586,7.903466,2.991328,14.386048,-0.949025,post-void,41
784,34.5,33.0,36.0,0.853115,0.669430,-0.025430,0.723724,3.582379,0.724171,0.523777,...,1.102943,1.787201,0.018320,0.403043,9.339641,5.014941,34.814527,-1.705910,post-void,41


## Data 8b
classes: pre-void, void, non-void

fs: original

overlap: 0.5

sliding window: 3s

filename: three_class_raw_3s_yo_0.5.csv

In [57]:
all_features = []
for exp_id, imu_data in enumerate(labelled_imu_dict.keys()):    
    # Extract features
    
    df = labelled_imu_dict[imu_data] 
    analyzer = GenerateFeatures(50, window_duration=3.0, overlap=0.5)
    features, _ = analyzer.analyze_multi_axis_imu(labelled_imu_dict[imu_data])

    table = analyzer.create_summary_table()
    table['experiment_id'] = exp_id + 1  # Track source
    all_features.append(table)
    

final_features = pd.concat(all_features, ignore_index=True)

Analyzing : 100%|██████████| 8/8 [00:00<00:00, 20.64it/s]


In [58]:
final_features.to_csv('/home/edumaba/Public/MPhil_Thesis/Code/wear_uropatch/feature_datasets/subject_based_norm/preprocessed/feature_set_2/without_gravity/three_class_pp_3s_0.5.csv', index=False)
final_features

,center_time,start_time,end_time,acc_x_permutation_entropy,acc_x_spectral_entropy,acc_x_mean,acc_x_std,acc_x_range,acc_x_rms,acc_x_var,...,gyr_mag_spectral_centroid,gyr_mag_spectral_spread,gyr_mag_spectral_flatness,gyr_mag_peak_frequency_ratio,gyr_mag_hjorth_frequency,gyr_mag_spectral_skewness,gyr_mag_spectral_kurtosis,gyr_mag_snr,label,experiment_id
0,1.5,0.0,3.0,0.810113,0.693263,-0.027271,1.542011,9.328774,1.542252,2.377798,...,2.217571,2.239953,0.023149,0.288816,6.824659,1.703263,8.248314,-3.913610,pre-void,1
1,3.0,1.5,4.5,0.861348,0.786883,-0.032372,0.501520,2.865148,0.502564,0.251522,...,2.423587,3.460616,0.106005,0.407713,8.475181,2.393441,9.513282,-1.621777,pre-void,1
2,4.5,3.0,6.0,0.860940,0.765878,-0.074015,0.781310,6.274071,0.784808,0.610446,...,2.473462,2.888966,0.062104,0.219981,7.693017,2.606673,11.146248,-5.497193,pre-void,1
3,6.0,4.5,7.5,0.867752,0.663605,-0.010304,1.427354,9.255942,1.427391,2.037340,...,1.986584,2.280573,0.039003,0.180937,8.459484,3.348351,18.618988,-6.557911,pre-void,1
4,7.5,6.0,9.0,0.864884,0.716930,0.159788,1.267820,9.255942,1.277850,1.607368,...,4.576898,3.476976,0.104631,0.098653,8.897261,1.152538,4.852549,-9.607808,pre-void,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1548,33.0,31.5,34.5,0.859992,0.681247,0.392837,0.862166,4.086704,0.947445,0.743330,...,1.155212,2.278543,0.029143,0.672876,8.287437,3.756834,19.066022,3.132218,post-void,41
1549,34.5,33.0,36.0,0.853115,0.669430,-0.025430,0.723724,3.582379,0.724171,0.523777,...,1.102943,1.787201,0.018320,0.403043,9.339641,5.014941,34.814527,-1.705910,post-void,41
1550,36.0,34.5,37.5,0.884058,0.752192,-0.091498,0.660839,3.473256,0.667143,0.436708,...,1.241497,2.637517,0.041096,0.653693,9.904417,4.308611,22.964720,2.759118,post-void,41
1551,37.5,36.0,39.0,0.872070,0.792822,0.078437,0.419835,1.881228,0.427099,0.176261,...,1.903847,2.340473,0.046692,0.200265,9.467239,3.461779,18.545315,-6.013401,post-void,41


## Data 8c
classes: pre-void, void, non-void

fs: original

overlap: 0.8

sliding window: 3s

filename: three_class_raw_3s_yo_0.8.csv

In [59]:
all_features = []
for exp_id, imu_data in enumerate(labelled_imu_dict.keys()):    
    # Extract features
    
    df = labelled_imu_dict[imu_data] 
    analyzer = GenerateFeatures(50, window_duration=3.0, overlap=0.8)
    features, _ = analyzer.analyze_multi_axis_imu(labelled_imu_dict[imu_data])

    table = analyzer.create_summary_table()
    table['experiment_id'] = exp_id + 1  # Track source
    all_features.append(table)
    

final_features = pd.concat(all_features, ignore_index=True)

Analyzing : 100%|██████████| 8/8 [00:00<00:00,  8.81it/s]


In [60]:
final_features.to_csv('/home/edumaba/Public/MPhil_Thesis/Code/wear_uropatch/feature_datasets/subject_based_norm/preprocessed/feature_set_2/without_gravity/three_class_pp_3s_0.8.csv', index=False)
final_features

,center_time,start_time,end_time,acc_x_permutation_entropy,acc_x_spectral_entropy,acc_x_mean,acc_x_std,acc_x_range,acc_x_rms,acc_x_var,...,gyr_mag_spectral_centroid,gyr_mag_spectral_spread,gyr_mag_spectral_flatness,gyr_mag_peak_frequency_ratio,gyr_mag_hjorth_frequency,gyr_mag_spectral_skewness,gyr_mag_spectral_kurtosis,gyr_mag_snr,label,experiment_id
0,1.5,0.0,3.0,0.810113,0.693263,-0.027271,1.542011,9.328774,1.542252,2.377798,...,2.217571,2.239953,0.023149,0.288816,6.824659,1.703263,8.248314,-3.913610,pre-void,1
1,2.1,0.6,3.6,0.838889,0.689832,0.132967,1.346999,9.328774,1.353546,1.814406,...,1.407672,1.922038,0.025778,0.504354,6.980160,3.261741,20.789511,0.075645,pre-void,1
2,2.7,1.2,4.2,0.841061,0.719862,0.081761,0.892067,6.167925,0.895806,0.795784,...,1.469831,2.913995,0.058483,0.739443,6.797415,3.483089,16.893739,4.530011,pre-void,1
3,3.3,1.8,4.8,0.868505,0.788763,0.038418,0.436185,2.367645,0.437874,0.190257,...,2.580828,3.158420,0.097709,0.235000,9.409347,2.369453,9.886240,-5.125924,pre-void,1
4,3.9,2.4,5.4,0.858704,0.753879,0.072001,0.457469,2.419192,0.463101,0.209278,...,2.770360,2.861699,0.092025,0.152028,9.144243,2.322410,9.807133,-7.464568,pre-void,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3844,36.9,35.4,38.4,0.889493,0.853034,0.125237,0.439363,2.160460,0.456863,0.193040,...,1.950152,2.472274,0.054490,0.229861,9.000922,3.368604,17.588053,-5.251036,post-void,41
3845,37.5,36.0,39.0,0.872070,0.792822,0.078437,0.419835,1.881228,0.427099,0.176261,...,1.903847,2.340473,0.046692,0.200265,9.467239,3.461779,18.545315,-6.013401,post-void,41
3846,38.1,36.6,39.6,0.888007,0.741409,0.008840,0.417610,1.866043,0.417704,0.174398,...,1.994368,2.711960,0.056730,0.243271,9.233634,3.214619,15.033282,-4.928499,post-void,41
3847,38.7,37.2,40.2,0.883981,0.712005,-0.026337,0.482329,1.972815,0.483047,0.232641,...,2.906138,3.959448,0.093973,0.373285,9.282671,2.026269,6.706598,-2.250295,post-void,41


## Data 9
classes: pre-void, void, non-void

fs: original

overlap: 0.0

sliding window: 4s

filename: three_class_raw_4s_no.csv

In [61]:
all_features = []
for exp_id, imu_data in enumerate(labelled_imu_dict.keys()):    
    # Extract features
    
    df = labelled_imu_dict[imu_data]
    analyzer = GenerateFeatures(50, window_duration=4.0, overlap=0.0)
    features, _ = analyzer.analyze_multi_axis_imu(labelled_imu_dict[imu_data])

    table = analyzer.create_summary_table()
    table['experiment_id'] = exp_id + 1  # Track source
    all_features.append(table)
    

final_features = pd.concat(all_features, ignore_index=True)

Analyzing :   0%|          | 0/8 [00:00<?, ?it/s]

Analyzing : 100%|██████████| 8/8 [00:00<00:00, 53.83it/s]


In [62]:
final_features.to_csv('/home/edumaba/Public/MPhil_Thesis/Code/wear_uropatch/feature_datasets/subject_based_norm/preprocessed/feature_set_2/without_gravity/three_class_pp_4s_no.csv', index=False)
final_features

,center_time,start_time,end_time,acc_x_permutation_entropy,acc_x_spectral_entropy,acc_x_mean,acc_x_std,acc_x_range,acc_x_rms,acc_x_var,...,gyr_mag_spectral_centroid,gyr_mag_spectral_spread,gyr_mag_spectral_flatness,gyr_mag_peak_frequency_ratio,gyr_mag_hjorth_frequency,gyr_mag_spectral_skewness,gyr_mag_spectral_kurtosis,gyr_mag_snr,label,experiment_id
0,2.0,0.0,4.0,0.817882,0.730478,-0.007210,1.358064,9.328774,1.358083,1.844337,...,1.557124,2.062926,0.021811,0.402585,6.908349,2.474312,13.097073,-1.714190,pre-void,1
1,6.0,4.0,8.0,0.873991,0.718701,0.073802,1.264717,9.255942,1.266869,1.599509,...,1.977269,2.379108,0.045955,0.186602,8.608010,3.192861,16.981391,-6.393864,pre-void,1
2,10.0,8.0,12.0,0.851945,0.780441,0.024442,0.760268,4.947701,0.760661,0.578007,...,1.825259,2.420545,0.044192,0.206155,8.420844,3.120383,14.579514,-5.855411,pre-void,1
3,14.0,12.0,16.0,0.887265,0.769384,0.002222,0.724263,5.867749,0.724266,0.524556,...,3.238903,4.060612,0.150389,0.164855,10.269913,2.398237,9.421729,-7.046582,pre-void,1
4,18.0,16.0,20.0,0.859675,0.564954,0.156076,1.214231,8.448298,1.224221,1.474358,...,1.501568,2.256641,0.038007,0.382938,7.247507,3.418449,18.719524,-2.071999,void,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
578,22.0,20.0,24.0,0.885593,0.849238,0.026771,0.387813,2.354625,0.388735,0.150399,...,3.920659,3.759766,0.134765,0.122551,9.762799,1.701635,5.962084,-8.549069,void,41
579,26.0,24.0,28.0,0.855799,0.799055,0.015270,0.633405,3.956391,0.633589,0.401202,...,1.915112,2.635405,0.048116,0.261175,9.120678,2.946976,14.061219,-4.516090,void,41
580,30.0,28.0,32.0,0.885225,0.669279,-0.154108,1.353756,6.699328,1.362499,1.832654,...,1.421260,2.032944,0.027265,0.261629,8.035978,3.943985,22.758930,-4.505892,post-void,41
581,34.0,32.0,36.0,0.862677,0.710413,0.150762,0.896947,4.949269,0.909529,0.804513,...,1.241865,2.158751,0.026591,0.416070,9.068352,4.015611,23.055747,-1.471948,post-void,41


## Data 9b
classes: pre-void, void, non-void

fs: original

overlap: 0.5

sliding window: 4s

filename: three_class_raw_4s_yo_0.5.csv

In [63]:
all_features = []
for exp_id, imu_data in enumerate(labelled_imu_dict.keys()):    
    # Extract features
    
    df = labelled_imu_dict[imu_data]
    analyzer = GenerateFeatures(50, window_duration=4.0, overlap=0.5)
    features, _ = analyzer.analyze_multi_axis_imu(labelled_imu_dict[imu_data])

    table = analyzer.create_summary_table()
    table['experiment_id'] = exp_id + 1  # Track source
    all_features.append(table)
    

final_features = pd.concat(all_features, ignore_index=True)

Analyzing : 100%|██████████| 8/8 [00:00<00:00, 20.58it/s]


In [64]:
final_features.to_csv('/home/edumaba/Public/MPhil_Thesis/Code/wear_uropatch/feature_datasets/subject_based_norm/preprocessed/feature_set_2/without_gravity/three_class_pp_4s_0.5.csv', index=False)
final_features

,center_time,start_time,end_time,acc_x_permutation_entropy,acc_x_spectral_entropy,acc_x_mean,acc_x_std,acc_x_range,acc_x_rms,acc_x_var,...,gyr_mag_spectral_centroid,gyr_mag_spectral_spread,gyr_mag_spectral_flatness,gyr_mag_peak_frequency_ratio,gyr_mag_hjorth_frequency,gyr_mag_spectral_skewness,gyr_mag_spectral_kurtosis,gyr_mag_snr,label,experiment_id
0,2.0,0.0,4.0,0.817882,0.730478,-0.007210,1.358064,9.328774,1.358083,1.844337,...,1.557124,2.062926,0.021811,0.402585,6.908349,2.474312,13.097073,-1.714190,pre-void,1
1,4.0,2.0,6.0,0.857991,0.795296,-0.067612,0.707095,6.274071,0.710320,0.499984,...,2.800656,2.940596,0.094825,0.123950,8.031590,2.357666,9.870134,-8.492838,pre-void,1
2,6.0,4.0,8.0,0.873991,0.718701,0.073802,1.264717,9.255942,1.266869,1.599509,...,1.977269,2.379108,0.045955,0.186602,8.608010,3.192861,16.981391,-6.393864,pre-void,1
3,8.0,6.0,10.0,0.870455,0.743484,0.088739,1.221992,9.255942,1.225210,1.493265,...,4.099706,3.335315,0.094136,0.117844,8.865780,1.348112,5.163787,-8.742374,pre-void,1
4,10.0,8.0,12.0,0.851945,0.780441,0.024442,0.760268,4.947701,0.760661,0.578007,...,1.825259,2.420545,0.044192,0.206155,8.420844,3.120383,14.579514,-5.855411,pre-void,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1142,30.0,28.0,32.0,0.885225,0.669279,-0.154108,1.353756,6.699328,1.362499,1.832654,...,1.421260,2.032944,0.027265,0.261629,8.035978,3.943985,22.758930,-4.505892,post-void,41
1143,32.0,30.0,34.0,0.870964,0.635406,-0.063167,1.359756,7.404909,1.361222,1.848936,...,0.953698,1.668268,0.011647,0.429380,8.149748,4.196951,25.955120,-1.235056,post-void,41
1144,34.0,32.0,36.0,0.862677,0.710413,0.150762,0.896947,4.949269,0.909529,0.804513,...,1.241865,2.158751,0.026591,0.416070,9.068352,4.015611,23.055747,-1.471948,post-void,41
1145,36.0,34.0,38.0,0.888686,0.751865,-0.033872,0.656173,3.582379,0.657046,0.430562,...,0.763123,1.826845,0.019007,0.654033,8.760944,6.400952,49.655048,2.765659,post-void,41


## Data 9c
classes: pre-void, void, non-void

fs: original

overlap: 0.8

sliding window: 4s

filename: three_class_raw_4s_yo_0.8.csv

In [65]:
all_features = []
for exp_id, imu_data in enumerate(labelled_imu_dict.keys()):    
    # Extract features
    
    df = labelled_imu_dict[imu_data]  
    analyzer = GenerateFeatures(50, window_duration=4.0, overlap=0.8)
    features, _ = analyzer.analyze_multi_axis_imu(labelled_imu_dict[imu_data])

    table = analyzer.create_summary_table()
    table['experiment_id'] = exp_id + 1  # Track source
    all_features.append(table)
    

final_features = pd.concat(all_features, ignore_index=True)

Analyzing : 100%|██████████| 8/8 [00:00<00:00, 10.21it/s]


In [66]:
final_features.to_csv('/home/edumaba/Public/MPhil_Thesis/Code/wear_uropatch/feature_datasets/subject_based_norm/preprocessed/feature_set_2/without_gravity/three_class_pp_4s_0.8.csv', index=False)
final_features

,center_time,start_time,end_time,acc_x_permutation_entropy,acc_x_spectral_entropy,acc_x_mean,acc_x_std,acc_x_range,acc_x_rms,acc_x_var,...,gyr_mag_spectral_centroid,gyr_mag_spectral_spread,gyr_mag_spectral_flatness,gyr_mag_peak_frequency_ratio,gyr_mag_hjorth_frequency,gyr_mag_spectral_skewness,gyr_mag_spectral_kurtosis,gyr_mag_snr,label,experiment_id
0,2.0,0.0,4.0,0.817882,0.730478,-0.007210,1.358064,9.328774,1.358083,1.844337,...,1.557124,2.062926,0.021811,0.402585,6.908349,2.474312,13.097073,-1.714190,pre-void,1
1,2.8,0.8,4.8,0.854140,0.749124,0.132066,1.184149,9.328774,1.191491,1.402208,...,1.206017,2.333514,0.044015,0.619105,7.023570,3.983487,23.177630,2.109588,pre-void,1
2,3.6,1.6,5.6,0.855774,0.810470,-0.022169,0.512968,2.916695,0.513446,0.263136,...,2.581682,2.999712,0.092733,0.154489,9.122141,2.376723,10.066417,-7.382219,pre-void,1
3,4.4,2.4,6.4,0.863030,0.745063,0.050930,0.980056,7.028317,0.981378,0.960509,...,2.385048,2.789759,0.068447,0.156583,8.120909,2.620646,11.614375,-7.312963,pre-void,1
4,5.2,3.2,7.2,0.870237,0.707849,0.014100,1.197028,9.255942,1.197111,1.432877,...,1.598819,2.095150,0.035222,0.244730,8.482186,3.612037,21.522176,-4.894157,pre-void,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2836,35.6,33.6,37.6,0.885050,0.722333,-0.022925,0.655185,3.582379,0.655585,0.429267,...,0.769778,1.656797,0.015811,0.471590,9.312566,6.944633,58.839325,-0.494057,post-void,41
2837,36.4,34.4,38.4,0.885270,0.771426,-0.023604,0.660690,3.582379,0.661111,0.436511,...,1.116999,2.210564,0.034004,0.611730,9.197958,4.567550,27.936967,1.974267,post-void,41
2838,37.2,35.2,39.2,0.889458,0.808835,0.084303,0.438863,2.198461,0.446887,0.192601,...,1.853206,2.373680,0.045924,0.157065,9.557931,3.468410,18.499539,-7.297143,post-void,41
2839,38.0,36.0,40.0,0.874288,0.805409,0.078048,0.417533,1.886429,0.424765,0.174333,...,1.967480,2.637047,0.052216,0.185446,9.328749,3.265261,15.727128,-6.427010,post-void,41


## Data 10
classes: pre-void, void, non-void

fs: original

overlap: 0.0

sliding window: 5s

filename: three_class_raw_5s_no.csv

In [67]:
all_features = []
for exp_id, imu_data in enumerate(labelled_imu_dict.keys()):    
    # Extract features
    
    df = labelled_imu_dict[imu_data]  
    analyzer = GenerateFeatures(50, window_duration=5.0, overlap=0.0)
    features, _ = analyzer.analyze_multi_axis_imu(labelled_imu_dict[imu_data])

    table = analyzer.create_summary_table()
    table['experiment_id'] = exp_id + 1  # Track source
    all_features.append(table)
    

final_features = pd.concat(all_features, ignore_index=True)

Analyzing : 100%|██████████| 8/8 [00:00<00:00, 28.07it/s]


In [68]:
final_features.to_csv('/home/edumaba/Public/MPhil_Thesis/Code/wear_uropatch/feature_datasets/subject_based_norm/preprocessed/feature_set_2/without_gravity/three_class_pp_5s_no.csv', index=False)
final_features

,center_time,start_time,end_time,acc_x_permutation_entropy,acc_x_spectral_entropy,acc_x_mean,acc_x_std,acc_x_range,acc_x_rms,acc_x_var,...,gyr_mag_spectral_centroid,gyr_mag_spectral_spread,gyr_mag_spectral_flatness,gyr_mag_peak_frequency_ratio,gyr_mag_hjorth_frequency,gyr_mag_spectral_skewness,gyr_mag_spectral_kurtosis,gyr_mag_snr,label,experiment_id
0,2.5,0.0,5.0,0.836178,0.735366,0.033616,1.227673,9.328774,1.228133,1.507182,...,1.293503,2.047922,0.025732,0.431386,6.998229,3.167231,18.329564,-1.199515,pre-void,1
1,7.5,5.0,10.0,0.861999,0.729198,-0.023397,1.220136,9.255942,1.220360,1.488732,...,3.826881,3.419829,0.098497,0.092278,8.581478,1.440256,5.554808,-9.928560,pre-void,1
2,12.5,10.0,15.0,0.868661,0.763908,0.088144,0.778410,5.867749,0.783385,0.605923,...,2.770559,3.225054,0.104721,0.195316,9.691982,2.826587,13.450387,-6.148881,pre-void,1
3,17.5,15.0,20.0,0.873451,0.838142,0.101102,1.104300,8.448298,1.108918,1.219479,...,1.868674,2.774330,0.052810,0.421165,7.688467,2.587741,11.439466,-1.381021,void,1
4,22.5,20.0,25.0,0.905193,0.791878,0.034429,0.527603,3.685213,0.528725,0.278365,...,3.449315,3.495680,0.114274,0.141525,9.103929,1.703707,6.649564,-7.828935,void,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458,17.5,15.0,20.0,0.919437,0.635725,-0.010074,0.575997,4.320962,0.576085,0.331772,...,3.266514,4.313718,0.135791,0.296209,9.829627,2.096473,7.424497,-3.758458,void,41
459,22.5,20.0,25.0,0.876903,0.845824,0.029246,0.399704,2.354625,0.400773,0.159763,...,2.621375,3.377868,0.086714,0.193630,9.571896,2.298301,9.228104,-6.195614,void,41
460,27.5,25.0,30.0,0.869104,0.791475,0.085675,0.816616,4.441176,0.821098,0.666861,...,2.920571,3.448432,0.082136,0.185931,8.890397,1.953727,7.120097,-6.413097,void,41
461,32.5,30.0,35.0,0.878172,0.585023,-0.056918,1.282186,7.404909,1.283449,1.644002,...,0.811423,1.617436,0.012523,0.386217,8.134003,4.831355,32.731497,-2.011829,post-void,41


## Data 10b
classes: pre-void, void, non-void

fs: original

overlap: 0.5

sliding window: 5s

filename: three_class_raw_5s_yo_0.5.csv

In [69]:
all_features = []
for exp_id, imu_data in enumerate(labelled_imu_dict.keys()):    
    # Extract features
    
    df = labelled_imu_dict[imu_data]
    analyzer = GenerateFeatures(50, window_duration=5.0, overlap=0.5)
    features, _ = analyzer.analyze_multi_axis_imu(labelled_imu_dict[imu_data])

    table = analyzer.create_summary_table()
    table['experiment_id'] = exp_id + 1  # Track source
    all_features.append(table)
    

final_features = pd.concat(all_features, ignore_index=True)

Analyzing : 100%|██████████| 8/8 [00:00<00:00, 27.58it/s]


In [70]:
final_features.to_csv('/home/edumaba/Public/MPhil_Thesis/Code/wear_uropatch/feature_datasets/subject_based_norm/preprocessed/feature_set_2/without_gravity/three_class_pp_5s_0.5.csv', index=False)
final_features

,center_time,start_time,end_time,acc_x_permutation_entropy,acc_x_spectral_entropy,acc_x_mean,acc_x_std,acc_x_range,acc_x_rms,acc_x_var,...,gyr_mag_spectral_centroid,gyr_mag_spectral_spread,gyr_mag_spectral_flatness,gyr_mag_peak_frequency_ratio,gyr_mag_hjorth_frequency,gyr_mag_spectral_skewness,gyr_mag_spectral_kurtosis,gyr_mag_snr,label,experiment_id
0,2.5,0.0,5.0,0.836178,0.735366,0.033616,1.227673,9.328774,1.228133,1.507182,...,1.293503,2.047922,0.025732,0.431386,6.998229,3.167231,18.329564,-1.199515,pre-void,1
1,5.0,2.5,7.5,0.867355,0.773011,0.027568,1.140440,9.255942,1.140773,1.300602,...,1.626706,2.188350,0.037821,0.202112,8.549961,3.483574,19.875325,-5.963493,pre-void,1
2,7.5,5.0,10.0,0.861999,0.729198,-0.023397,1.220136,9.255942,1.220360,1.488732,...,3.826881,3.419829,0.098497,0.092278,8.581478,1.440256,5.554808,-9.928560,pre-void,1
3,10.0,7.5,12.5,0.864162,0.769046,0.062258,0.727212,4.947701,0.729872,0.528837,...,1.845261,2.476466,0.044373,0.183175,8.491505,3.116145,14.653196,-6.492619,pre-void,1
4,12.5,10.0,15.0,0.868661,0.763908,0.088144,0.778410,5.867749,0.783385,0.605923,...,2.770559,3.225054,0.104721,0.195316,9.691982,2.826587,13.450387,-6.148881,pre-void,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
900,27.5,25.0,30.0,0.869104,0.791475,0.085675,0.816616,4.441176,0.821098,0.666861,...,2.920571,3.448432,0.082136,0.185931,8.890397,1.953727,7.120097,-6.413097,void,41
901,30.0,27.5,32.5,0.873447,0.673044,-0.004083,1.360501,7.404909,1.360507,1.850962,...,1.415054,2.129289,0.027586,0.320156,8.032693,3.527754,18.830806,-3.270467,post-void,41
902,32.5,30.0,35.0,0.878172,0.585023,-0.056918,1.282186,7.404909,1.283449,1.644002,...,0.811423,1.617436,0.012523,0.386217,8.134003,4.831355,32.731497,-2.011829,post-void,41
903,35.0,32.5,37.5,0.886560,0.693911,-0.023923,0.625897,3.582379,0.626354,0.391747,...,0.800765,1.593869,0.014678,0.320832,9.330338,6.377293,52.936565,-3.256988,post-void,41


## Data 10c
classes: pre-void, void, non-void

fs: original

overlap: 0.8

sliding window: 5s

filename: three_class_raw_5s_yo_0.8.csv

In [71]:
all_features = []
for exp_id, imu_data in enumerate(labelled_imu_dict.keys()):    
    # Extract features
    
    df = labelled_imu_dict[imu_data]
    analyzer = GenerateFeatures(50, window_duration=5.0, overlap=0.8)
    features, _ = analyzer.analyze_multi_axis_imu(labelled_imu_dict[imu_data])

    table = analyzer.create_summary_table()
    table['experiment_id'] = exp_id + 1  # Track source
    all_features.append(table)
    

final_features = pd.concat(all_features, ignore_index=True)

Analyzing : 100%|██████████| 8/8 [00:00<00:00, 14.12it/s]


In [72]:
final_features.to_csv('/home/edumaba/Public/MPhil_Thesis/Code/wear_uropatch/feature_datasets/subject_based_norm/preprocessed/feature_set_2/without_gravity/three_class_pp_5s_0.8.csv', index=False)
final_features

,center_time,start_time,end_time,acc_x_permutation_entropy,acc_x_spectral_entropy,acc_x_mean,acc_x_std,acc_x_range,acc_x_rms,acc_x_var,...,gyr_mag_spectral_centroid,gyr_mag_spectral_spread,gyr_mag_spectral_flatness,gyr_mag_peak_frequency_ratio,gyr_mag_hjorth_frequency,gyr_mag_spectral_skewness,gyr_mag_spectral_kurtosis,gyr_mag_snr,label,experiment_id
0,2.5,0.0,5.0,0.836178,0.735366,0.033616,1.227673,9.328774,1.228133,1.507182,...,1.293503,2.047922,0.025732,0.431386,6.998229,3.167231,18.329564,-1.199515,pre-void,1
1,3.5,1.0,6.0,0.852309,0.791830,0.020127,0.945915,6.721387,0.946129,0.894755,...,1.774867,2.734964,0.063152,0.391094,6.924517,2.919970,13.685695,-1.922683,pre-void,1
2,4.5,2.0,7.0,0.867167,0.802191,0.060185,0.949502,7.028317,0.951407,0.901553,...,1.868768,2.462367,0.046712,0.220731,8.741791,3.066202,15.448542,-5.478233,pre-void,1
3,5.5,3.0,8.0,0.871984,0.726645,0.069636,1.152173,9.255942,1.154275,1.327502,...,1.715215,2.228004,0.035930,0.190328,8.613046,3.421732,19.335728,-6.288072,pre-void,1
4,6.5,4.0,9.0,0.867709,0.711022,0.040869,1.139841,9.255942,1.140573,1.299237,...,2.424572,2.807033,0.062838,0.134746,8.596368,2.526069,11.284498,-8.076293,pre-void,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2229,34.5,32.0,37.0,0.871768,0.700654,0.171792,0.813760,4.949269,0.831696,0.662205,...,0.917056,1.749123,0.017671,0.309577,9.123578,5.429247,39.678669,-3.483457,post-void,41
2230,35.5,33.0,38.0,0.876980,0.703722,-0.008924,0.623014,3.582379,0.623078,0.388146,...,0.748426,1.582060,0.014971,0.365689,9.085512,6.878526,59.494460,-2.391903,post-void,41
2231,36.5,34.0,39.0,0.889697,0.693639,0.013631,0.617299,3.582379,0.617450,0.381059,...,1.241208,2.192690,0.031610,0.370148,9.191728,4.322373,26.146442,-2.308636,post-void,41
2232,37.5,35.0,40.0,0.883830,0.810996,0.035391,0.499983,2.576756,0.501234,0.249983,...,1.892710,2.494654,0.047245,0.144326,9.502806,3.388598,17.239690,-7.729651,post-void,41
